# Cognitive Modeling - Assignment 2
Yorick Juffer \
s1993623

## Libraries

In [52]:
import math
import random
from model import Model
from dmchunk import Chunk

import numpy as np

## Functions

In [53]:
t_0 = 0.011
a = 1.1
b = 0.015
add_noise = True

def noise(s):
    rand = random.uniform(0.001, 0.999)
    return s * math.log((1 - rand)/rand)

def time_to_pulses(time, t_0 = t_0, a = a, b = b, add_noise = add_noise):  
    pulses = 0
    pulse_duration = t_0
    
    while time >= pulse_duration:
        time = time - pulse_duration
        pulses = pulses + 1
        pulse_duration = a * pulse_duration + add_noise * noise(b * a * pulse_duration)
        
    return pulses

def pulses_to_time(pulses, t_0 = t_0, a = a, b = b, add_noise = add_noise):
    time = 0
    pulse_duration = t_0
    
    while pulses > 0:
        time = time + pulse_duration
        pulses = pulses - 1
        pulse_duration = a * pulse_duration + add_noise * noise(b * a * pulse_duration)
    
    return time

## Experiment

In [54]:
def experiment():

    intertrial_interval = []
    reward_visibility = [
        [0, 0], 
        [1, 0], 
        [1, 1], 
        [0, 1]]
    foreperiod_location = [
        [0.3, 0],
        [0.3, 1],
        [0.6, 0],
        [0.6, 1],
        [0.9, 0],
        [0.9, 1]]

    participant_model = Model()
    count = 0

    for block in range(16):

        index_block = 0
        if(index_block == 3):
            random.shuffle(reward_visibility)
            index_block = 0
        
        for trial in range(30):

            index_trial = 0
            if(index_trial == 4):
                random.shuffle(foreperiod_location)
                index_trial = 0


            

    return

## Run the Experiment

In [55]:
experiment()

## Data Analysis